In [ ]:
from Assignment_Queries.pandas_lib.std_imports import *
from Assignment_Queries.pandas_lib.func_lib import *
from Server_Cleaning.modules.Assignment_datasets import *
path = '../'

# Introduction
In this document, we want to perform analyses that provide information about **football clubs**. 
We will pay particular attention to the average goals scored/conceded.
Furthermore, we will focus on one club (of our choice) and analyse it.

In [ ]:
# Getting and cleaning the DataFrames
clubs = clear_clubs(get_clubs(path))
club_games = clear_club_games(get_club_games(path))
games = clear_games(get_games(path))
game_events = clear_game_events(get_game_events(path))